## Machine Learning - Árvore de Decisão | Regressão Logistica | Random Florest usando Stratified K-Fold e avaliando a métrica AUC na base Out of Time

In [ ]:
# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
run 'path_dataset'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_orders.shape

(492138, 43)

In [ ]:
!pip install feature-engine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 276 kB 4.3 MB/s 
     |████████████████████████████████| 9.8 MB 57.7 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
# Importando as bibliotecas
import numpy as np
import pandas as pd
# from datetime import date
import numpy as np
# from sklearn.model_selection import train_test_split
import time

## Função para avaliar as técnicas

In [ ]:
# Função para medir a acurácia
import numpy as np
from sklearn.metrics import accuracy_score
def avaliar_tecnica(cv_results):
  mean_accuracy = np.array(cv_results).mean()
  std_accuracy  = np.array(cv_results).std()
  print(f"Acurácia Média com CV: {100 * mean_accuracy:.3f}%")
  print(f"Desvio-Padrão com CV: {100 * std_accuracy:.3f}%")

## Identificar as variáveis de modelagem

In [ ]:
# Variáveis chaves da tabela
key_vars = ['customer_id', 'data_ref']

# Variáveis numéricas
num_vars = ['receita_1m', 'qtd_pedidos_1m', 'recencia']

# Variáveis categorica
# cat_vars = ['customer_id']

# Variável resposta/target
target = 'churn_next_month'

# criando a lista com as features
features = num_vars

# Filtrando a base de features
X_train = df_abt_train[features]

# Filtrando o target
y_train = df_abt_train[target]

# Filtrando a base de features
X_oot = df_abt_oot[features]

# Filtrando o target
y_oot = df_abt_oot[target]  

## Verificando as coluna X_train e y_train

In [ ]:
# Verificando o X e as suas colunas
print(f'Verificando shape   de X_train: {X_train.shape}')
print(f'Verificando colunas de X_train: {X_train.columns.to_list()}')
print('---------------------------------------------------')
print(f'Verificando shape   de y_train: {y_train.shape}')
print('---------------------------------------------------')
print(f'Verificando shape   de X_oot: {X_oot.shape}')
print(f'Verificando colunas de X_oot: {X_oot.columns.to_list()}')
print('---------------------------------------------------')
print(f'Verificando shape   de y_oot: {y_oot.shape}')
print('---------------------------------------------------')

Verificando shape   de X_train: (30079, 3)
Verificando colunas de X_train: ['receita_1m', 'qtd_pedidos_1m', 'recencia']
---------------------------------------------------
Verificando shape   de y_train: (30079,)
---------------------------------------------------
Verificando shape   de X_oot: (18441, 3)
Verificando colunas de X_oot: ['receita_1m', 'qtd_pedidos_1m', 'recencia']
---------------------------------------------------
Verificando shape   de y_oot: (18441,)
---------------------------------------------------


## Importando Bibliotecas

In [ ]:
# Importando as bibliotecas
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold # Validação Cross

from sklearn.metrics import roc_auc_score # Avaliando o modelo
from sklearn.model_selection import cross_val_score # Cross Validation

from sklearn.preprocessing import StandardScaler # Feature Scaling
from feature_engine.wrappers import SklearnTransformerWrapper

from feature_engine.imputation import CategoricalImputer, ArbitraryNumberImputer # Para preencher os dados caso venha faltando, Analise Exploratória
from feature_engine.encoding import OneHotEncoder # Cria uma nova coluna booleana para cada valor da variável categórica.

In [ ]:
# Random
random_state = 42

# Treinando DecisionTreeClassifier - Árvore de Decisão

In [ ]:
# Importando algoritmo de Árvore de Decisão
from sklearn.tree import DecisionTreeClassifier 

dt_model = Pipeline(steps=[
    ('numeric_imputer', ArbitraryNumberImputer(variables=num_vars, arbitrary_number=-999)), # Preenche os valores numericos faltantes
    # ('categoric_imputer', CategoricalImputer(variables=cat_vars, fill_value='missing')), # Preenche os valores categoricos faltantes  
    # ('one_hot_encoder', OneHotEncoder(variables=cat_vars)), # Transforma as variáveis categoricas para numericas 
    ('Decision_Tree', DecisionTreeClassifier(random_state=random_state)), # Modelo de Machine Learning - Árvore de Decisão
])

## StratifiedKFold

In [ ]:
# Criando o objeto StratifiedKFold com k=5, aleatorizando as linhas antes do split com shuffle=True
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

In [ ]:
# Cria uma nova coluna nos dados de treino que irá armazenar o valor 
# da nova coluna que guardará o valor a qual fold pertence cada linha
X_train["skfold"] = -1

# skf.split() retorna os índices de cada fold. 
# Criando o conjunto de treino e validação.
# Fazemos uma iteração sobre cada fold e atribuímos a cada linha qual fold ela pertence
for fold, (trn_, val_) in enumerate(skf.split(X=X_train, y=y_train)):
    X_train.loc[val_, 'skfold'] = fold

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
# Verificando o dataset
X_train.head(3)

,receita_1m,qtd_pedidos_1m,recencia
0,102.0,2,15
1,52.0,1,19
2,59.3,1,29


In [ ]:
# Distribuição da variável target em cada fold.
pd.crosstab(X_train['skfold'], y_train, normalize='index')

churn_next_month,0,1
skfold,,
0,0.613045,0.386955
1,0.613106,0.386894
2,0.613106,0.386894


In [ ]:
# Distribuição da variável target
df_abt_train['churn_next_month'].value_counts(1)

0    0.613086
1    0.386914
Name: churn_next_month, dtype: float64

## Avaliando o modelo na base de treino

In [ ]:
# Definindo uma estratégia de StratifiedKFold com k=5
# cross_val_score - estima a precisão esperada do seu modelo em dados fora do treinamento
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

# Tempo inicial
ti = time.time()

cv_results = cross_val_score(estimator=dt_model, X=X_train, y=y_train, scoring='roc_auc', cv=skf, n_jobs=-1)

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

Demorou 0.3784799575805664 segundos


In [ ]:
# Medindo a acurácia do algoritmo
avaliar_tecnica(cv_results)

Acurácia Média com CV: 61.475%
Desvio-Padrão com CV: 1.130%


## Avaliando o modelo na base OOT

In [ ]:
# Avaliando o modelo na base Out of Time
# roc_auc_score - métricas de classificação nos dados de treino e teste
# Tempo inicial
ti = time.time()

dt_model.fit(X_train, y_train)

y_pred_train = dt_model.predict(X_train)
y_pred_oot   = dt_model.predict(X_oot)

acc_train = roc_auc_score(y_train, y_pred_train)
acc_oot   = roc_auc_score(y_oot, y_pred_oot)

print(f'Acurácia Treino: {100 * acc_train:.3f}%')
print(f'Acurácia Teste:  {100 * acc_oot:.3f}%')

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

Acurácia Treino: 91.763%
Acurácia Teste:  57.437%
Demorou 0.13396620750427246 segundos


## Calculando a probabilidade, predict_prob, na base de Treino e na OOT

In [ ]:
# Calculando a predict_prob
dt_model.fit(X_train, y_train)
print(f"ROCAUC (Treino): {100 * roc_auc_score(y_train, dt_model.predict_proba(X_train)[:,1]):.3f}%")
print(f"ROCAUC (Teste): {100 * roc_auc_score(y_oot, dt_model.predict_proba(X_oot)[:,1]):.3f}%")

ROCAUC (Treino): 98.795%
ROCAUC (Teste): 59.422%


## Grid Search - GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

# Criamos um dicionário que os hiperparâmetros que queremos treinar o modelo
# Deve ser o mesmo nome do Pipeline e são dois undelines
parameters = {
  # Profundidade da árvore.
  'Decision_Tree__max_depth': [3, 5, 7, 9],
  # Função de separação da árvore. Valores possíveis: gini e entropy.
  'Decision_Tree__criterion': ['gini', 'entropy'],
  # pesos das classes em um dicionário. O valor balanced definirá valores na proporção inversa das frequências das classes. Possíveis valores: balanced e None.
  'Decision_Tree__class_weight': ['balanced', None],
  # Possíveis valores
  'Decision_Tree__min_samples_split': [2],
}

# Criamos um GridSearch passando o modelo, os parâmetros, a métrica que queremos otimizar. 
grid_search = GridSearchCV(dt_model, parameters, scoring='roc_auc', cv=5, n_jobs=-1)

# Realiza o treinamento com os dados
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('numeric_imputer',
                                        ArbitraryNumberImputer(arbitrary_number=-999,
                                                               variables=['receita_1m',
                                                                          'qtd_pedidos_1m',
                                                                          'recencia'])),
                                       ('Decision_Tree',
                                        DecisionTreeClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'Decision_Tree__class_weight': ['balanced', None],
                         'Decision_Tree__criterion': ['gini', 'entropy'],
                         'Decision_Tree__max_depth': [3, 5, 7, 9],
                         'Decision_Tree__min_samples_split': [2]},
             scoring='roc_auc')

In [ ]:
# cv_results_ - é um resultado
grid_search.cv_results_

{'mean_fit_time': array([0.04169827, 0.05616379, 0.06975694, 0.0727963 , 0.04855814,
        0.06514125, 0.08060842, 0.09759102, 0.03169208, 0.04516683,
        0.05795016, 0.06087027, 0.04341021, 0.0534236 , 0.06741238,
        0.07768908]),
 'mean_score_time': array([0.01151423, 0.01132674, 0.0101346 , 0.01075249, 0.00991421,
        0.00956187, 0.01089449, 0.00995665, 0.00975327, 0.0101264 ,
        0.01011391, 0.01020513, 0.01279254, 0.00962954, 0.01190095,
        0.00959468]),
 'mean_test_score': array([0.7453268 , 0.75200447, 0.74934691, 0.73980815, 0.74390151,
        0.75129462, 0.75054866, 0.74016236, 0.74402551, 0.7512842 ,
        0.74837622, 0.73667784, 0.74390471, 0.75137529, 0.74981126,
        0.74070054]),
 'param_Decision_Tree__class_weight': masked_array(data=['balanced', 'balanced', 'balanced', 'balanced',
                    'balanced', 'balanced', 'balanced', 'balanced', None,
                    None, None, None, None, None, None, None],
              mask=[False

In [ ]:
# Criando uma tabela com os resultados do grid_search
results = pd.DataFrame(grid_search.cv_results_)

# rankeando esses resultados
results.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_Decision_Tree__class_weight,param_Decision_Tree__criterion,param_Decision_Tree__max_depth,param_Decision_Tree__min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,0.056164,0.010316,0.011327,0.002574,balanced,gini,5,2,"{'Decision_Tree__class_weight': 'balanced', 'D...",0.748241,0.748831,0.758230,0.749135,0.755584,0.752004,0.004100,1
13,0.053424,0.005637,0.009630,0.000296,None,entropy,5,2,"{'Decision_Tree__class_weight': None, 'Decisio...",0.747532,0.748345,0.757352,0.748199,0.755449,0.751375,0.004156,2
5,0.065141,0.004772,0.009562,0.000277,balanced,entropy,5,2,"{'Decision_Tree__class_weight': 'balanced', 'D...",0.748785,0.748089,0.756463,0.748200,0.754936,0.751295,0.003637,3
9,0.045167,0.002678,0.010126,0.000431,None,gini,5,2,"{'Decision_Tree__class_weight': None, 'Decisio...",0.749632,0.747032,0.755868,0.748004,0.755885,0.751284,0.003841,4
6,0.080608,0.007591,0.010894,0.002894,balanced,entropy,7,2,"{'Decision_Tree__class_weight': 'balanced', 'D...",0.745478,0.747549,0.755150,0.749860,0.754706,0.750549,0.003837,5
14,0.067412,0.003485,0.011901,0.002970,None,entropy,7,2,"{'Decision_Tree__class_weight': None, 'Decisio...",0.744549,0.745877,0.754641,0.749689,0.754300,0.749811,0.004163,6
2,0.069757,0.010764,0.010135,0.000451,balanced,gini,7,2,"{'Decision_Tree__class_weight': 'balanced', 'D...",0.744692,0.742898,0.753466,0.750918,0.754760,0.749347,0.004733,7
10,0.057950,0.006679,0.010114,0.000956,None,gini,7,2,"{'Decision_Tree__class_weight': None, 'Decisio...",0.741827,0.742430,0.753072,0.749302,0.755250,0.748376,0.005448,8
0,0.041698,0.004303,0.011514,0.003659,balanced,gini,3,2,"{'Decision_Tree__class_weight': 'balanced', 'D...",0.740028,0.742749,0.752359,0.740202,0.751296,0.745327,0.005405,9
8,0.031692,0.000873,0.009753,0.000890,None,gini,3,2,"{'Decision_Tree__class_weight': None, 'Decisio...",0.741659,0.742454,0.748562,0.739358,0.748095,0.744026,0.003660,10


# Treinando LogisticRegression - Regressão Logistica

In [ ]:
from sklearn.linear_model import LogisticRegression # Regressão Linear

lr_model = Pipeline(steps=[
    ('numeric_imputer', ArbitraryNumberImputer(variables=num_vars, arbitrary_number=-999)), # Preenche os valores numericos faltantes
    # ('categoric_imputer', CategoricalImputer(variables=cat_vars, fill_value='missing')), # Preenche os valores categoricos faltantes 
    ('std', SklearnTransformerWrapper(transformer=StandardScaler(), variables=num_vars)), # Coloca em escala as variáveis
    # ('one_hot_encoder', OneHotEncoder(variables=cat_vars)), # Transforma as variáveis categoricas para numericas 
    ('Logistic_Regression', LogisticRegression(random_state=random_state)), # Modelo de Machine Learning - Árvore de Decisão
])

## StratifiedKFold

In [ ]:
# Criando o objeto StratifiedKFold com k=5, aleatorizando as linhas antes do split com shuffle=True
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

In [ ]:
# Cria uma nova coluna nos dados de treino que irá armazenar o valor 
# da nova coluna que guardará o valor a qual fold pertence cada linha
X_train["skfold"] = -1

# skf.split() retorna os índices de cada fold. 
# Criando o conjunto de treino e validação.
# Fazemos uma iteração sobre cada fold e atribuímos a cada linha qual fold ela pertence
for fold, (trn_, val_) in enumerate(skf.split(X=X_train, y=y_train)):
    X_train.loc[val_, 'skfold'] = fold

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
# Verificando o dataset
X_train.head(3)

,receita_1m,qtd_pedidos_1m,recencia,customer_id,skfold
0,102.0,2,15,0001a8e61d8b08ad436e8e6f4adeb399b88df962c72d9d...,3
1,52.0,1,19,0001a9f97d01d2696cf70c7657ee2d039388d691720ff9...,2
2,59.3,1,29,0004720dc16aed1f98fd79f59736170e0d686199cd9ae5...,4


In [ ]:
# Distribuição da variável target em cada fold.
pd.crosstab(X_train['skfold'], y_train, normalize='index')

churn_next_month,0,1
skfold,,
0,0.613032,0.386968
1,0.613032,0.386968
2,0.613032,0.386968
3,0.613198,0.386802
4,0.613134,0.386866


## Avaliando o modelo na base de treino

In [ ]:
# Definindo uma estratégia de StratifiedKFold com k=5
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Tempo inicial
ti = time.time()

cv_results = cross_val_score(estimator=lr_model, X=X_train, y=y_train, scoring='roc_auc', cv=skf, n_jobs=-1)

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

Demorou 0.312122106552124 segundos


In [ ]:
# Medindo a acurácia e o desvio padrão do algoritmo
avaliar_tecnica(cv_results)

Acurácia Média com CV: 74.936%
Desvio-Padrão com CV: 0.210%


## Avaliando o modelo na base OOT

In [ ]:
# Avaliando o modelo na base Out of Time
ti = time.time()

lr_model.fit(X_train, y_train)

y_pred_train = lr_model.predict(X_train)
y_pred_oot   = lr_model.predict(X_oot)

acc_train = roc_auc_score(y_train, y_pred_train)
acc_oot   = roc_auc_score(y_oot, y_pred_oot)

print(f'Acurácia Treino: {100 * acc_train:.3f}%')
print(f'Acurácia Teste:  {100 * acc_oot:.3f}%')

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

Acurácia Treino: 66.729%
Acurácia Teste:  64.667%
Demorou 0.13601374626159668 segundos


## Calculando a probabilidade, predict_proba, na base de Treino e na OOT

In [ ]:
# Calculando a predict_prob
lr_model.fit(X_train, y_train)
print(f"ROCAUC (Treino): {100 * roc_auc_score(y_train, lr_model.predict_proba(X_train)[:,1]):.3f}%")
print(f"ROCAUC (Teste): {100 * roc_auc_score(y_oot, lr_model.predict_proba(X_oot)[:,1]):.3f}%")

ROCAUC (Treino): 74.950%
ROCAUC (Teste): 73.600%


## Grid Search - GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

# Criamos um dicionário que os hiperparâmetros que queremos treinar o modelo
# Deve ser o mesmo nome do Pipeline e são dois undelines
parameters = {
  # Norma de penalização none e l2.
  'Logistic_Regression__penalty': ['none', 'l2'],
  # Ponto flutuante positivo. Inverso da força de regularização. Um valor menor significa uma regularização mais forte. Possíveis valores: 0.001, 0.01, 0.1, 1, 10, 100,
  'Logistic_Regression__C': [0.001, 0.01, 0.1, 1, 10, 100],
  # pesos das classes em um dicionário. O valor balanced definirá valores na proporção inversa das frequências das classes. Possíveis valores: balanced e None.
  'Logistic_Regression__class_weight': ['balanced', None],
  # Adiciona viés (bias) à função de decisão. Possíveis valores: True/False.
  'Logistic_Regression__fit_intercept': [True, False]
}

# Criamos um GridSearch passando o modelo, os parâmetros, a métrica que queremos otimizar. 
grid_search = GridSearchCV(lr_model, parameters, scoring='roc_auc', cv=5, n_jobs=-1)

# Realiza o treinamento com os dados
grid_search.fit(X_train, y_train)

In [ ]:
# cv_results_ - é um resultado
grid_search.cv_results_

In [ ]:
# criando uma tabela com os resultados do grid_search
results = pd.DataFrame(grid_search.cv_results_)

# rankeando esses resultados
results.sort_values(by='rank_test_score')

# Treinando RandomForestClassifier - Random Florest

In [ ]:
from sklearn.ensemble import RandomForestClassifier # Random Florest

rf_model = Pipeline(steps=[
    ('numeric_imputer', ArbitraryNumberImputer(variables=num_vars, arbitrary_number=-999)), # Preenche os valores numericos faltantes
    # ('categoric_imputer', CategoricalImputer(variables=cat_vars, fill_value='missing')), # Preenche os valores categoricos faltantes  
    # ('one_hot_encoder', OneHotEncoder(variables=cat_vars)),  
    ('Random_Forest_Classifier', RandomForestClassifier(random_state=random_state)), # Modelo de Machine Learning - Árvore de Decisão
])

## StratifiedKFold

In [ ]:
# Criando o objeto StratifiedKFold com k=5, aleatorizando as linhas antes do split com shuffle=True
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=random_state)

In [ ]:
# Cria uma nova coluna nos dados de treino que irá armazenar o valor 
# da nova coluna que guardará o valor a qual fold pertence cada linha
X_train["skfold"] = -1

# skf.split() retorna os índices de cada fold. 
# Criando o conjunto de treino e validação.
# Fazemos uma iteração sobre cada fold e atribuímos a cada linha qual fold ela pertence
for fold, (trn_, val_) in enumerate(skf.split(X=X_train, y=y_train)):
    X_train.loc[val_, 'skfold'] = fold

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
# Distribuição da variável target em cada fold.
pd.crosstab(X_train['skfold'], y_train, normalize='index')

churn_next_month,0,1
skfold,,
0,0.613032,0.386968
1,0.613032,0.386968
2,0.613032,0.386968
3,0.613198,0.386802
4,0.613134,0.386866


## Avaliando o modelo na base de treino

In [ ]:
# Definindo uma estratégia de StratifiedKFold com k=5
# scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Tempo inicial
ti = time.time()

cv_results = cross_val_score(estimator=rf_model, X=X_train, y=y_train, scoring='roc_auc', cv=skf, n_jobs=-1)

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

Demorou 10.214788913726807 segundos


In [ ]:
# Medindo a acurácia do algoritmo
avaliar_tecnica(cv_results)

Acurácia Média com CV: 67.773%
Desvio-Padrão com CV: 0.770%


## Avaliando o modelo na base OOT

In [ ]:
# Avaliando o modelo na base Out of Time
ti = time.time()

rf_model.fit(X_train, y_train)

y_pred_train = rf_model.predict(X_train)
y_pred_oot   = rf_model.predict(X_oot)

acc_train = roc_auc_score(y_train, y_pred_train)
acc_oot   = roc_auc_score(y_oot, y_pred_oot)

print(f'Acurácia Treino: {100 * acc_train:.3f}%')
print(f'Acurácia Teste:  {100 * acc_oot:.3f}%')

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

Acurácia Treino: 92.691%
Acurácia Teste:  59.533%
Demorou 4.266194581985474 segundos


## Calculando a probabilidade, predict_proba, na base de Treino e na OOT

In [ ]:
# Calculando a predict_prob
rf_model.fit(X_train, y_train)
print(f"ROCAUC (Treino): {100 * roc_auc_score(y_train, rf_model.predict_proba(X_train)[:,1]):.3f}%")
print(f"ROCAUC (Teste): {100 * roc_auc_score(y_oot, rf_model.predict_proba(X_oot)[:,1]):.3f}%")

ROCAUC (Treino): 98.150%
ROCAUC (Teste): 66.348%


## Grid Search - GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

ti = time.time()
# Criamos um dicionário que os hiperparâmetros que queremos treinar o modelo
# Deve ser o mesmo nome do Pipeline e são dois undelines
parameters = {
  # Profundidade da árvore
  'Random_Forest_Classifier__max_depth': [3, 5, 7, 9], # 25, 30, None],
  # Número de árvores na floresta aleatório
  'Random_Forest_Classifier__n_estimators': [120, 300], # 500, 800, 1200],
  # Número de atributos (características) para analisar na separação.
  'Random_Forest_Classifier__max_features': ['log2'], # 'sqrt', 'None'],
  # Número mínimo de amostras exigido para separar um nó. Valor padrão é 2. Possíveis valores: 2, 5, 10, 15, 100.
  'Random_Forest_Classifier__min_samples_split': [2, 5], # 10, 15, 100],
  # Número mínimo de amostras em cada folha. Valor padrão é 1. Possíveis valores: 1, 2, 5, 10.
  'Random_Forest_Classifier__min_samples_leaf': [1, 2], # 5, 10],
  # Pesos das classes em um dicionário. O valor balanced definirá valores na proporção inversa das frequências das classes. Possíveis valores: balanced e None.
  'Random_Forest_Classifier__class_weight': ['balanced', None]
}

# Criamos um GridSearch passando o modelo, os parâmetros, a métrica que queremos otimizar. 
grid_search = GridSearchCV(rf_model, parameters, scoring='roc_auc', cv=5, n_jobs=-1)

# Realiza o treinamento com os dados
grid_search.fit(X_train, y_train)

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

In [ ]:
# cv_results_ - é um resultado
grid_search.cv_results_

In [ ]:
# criando uma tabela com os resultados do grid_search
results = pd.DataFrame(grid_search.cv_results_)

# rankeando esses resultados
results.sort_values(by='rank_test_score')

# Treinando LGBM

In [ ]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from lightgbm import LGBMClassifier
lgbm_model = Pipeline(steps=[
    ('numeric_imputer', ArbitraryNumberImputer(variables=num_vars, arbitrary_number=-999)),
    # ('categoric_imputer', CategoricalImputer(variables=cat_vars, return_object=True)),
    # ('one_hot_encoder', OneHotEncoder(variables=cat_vars)),
    ('LGBM_Classifier', LGBMClassifier(random_state=random_state))
])

## StratifiedKFold

In [ ]:
# Criando o objeto StratifiedKFold com k=5, aleatorizando as linhas antes do split com shuffle=True
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Cria uma nova coluna nos dados de treino que irá armazenar o valor 
# da nova coluna que guardará o valor a qual fold pertence cada linha
X_train["skfold"] = -1

# skf.split() retorna os índices de cada fold. 
# Criando o conjunto de treino e validação.
# Fazemos uma iteração sobre cada fold e atribuímos a cada linha qual fold ela pertence
for fold, (trn_, val_) in enumerate(skf.split(X=X_train, y=y_train)):
    X_train.loc[val_, 'skfold'] = fold

In [ ]:
# Verificando o dataset
X_train.head(3)

In [ ]:
# Distribuição da variável target em cada fold.
pd.crosstab(X_train['skfold'], y_train, normalize='index')

## Avaliando o modelo na base de treino

In [ ]:
# Definindo uma estratégia de StratifiedKFold com k=5
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)


# Tempo inicial
ti = time.time()

cv_results = cross_val_score(estimator=lgbm_model, X=X_train, y=y_train, scoring='roc_auc', cv=skf, n_jobs=-1)

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

Demorou 1.1114544868469238 segundos


In [ ]:
# Medindo a acurácia na base de treino
avaliar_tecnica(cv_results)

Acurácia Média com CV: 75.193%
Desvio-Padrão com CV: 0.500%


##  Avaliando o modelo na base OOT - Out of Time

In [ ]:
# Avaliando o modelo na base OOT - Out of Time
ti = time.time()

lgbm_model.fit(X_train, y_train)

y_pred_train = lgbm_model.predict(X_train)
y_pred_oot   = lgbm_model.predict(X_oot)

acc_train = roc_auc_score(y_train, y_pred_train)
acc_oot   = roc_auc_score(y_oot, y_pred_oot)

print(f'Acurácia Treino: {100 * acc_train:.3f}%')
print(f'Acurácia Teste:  {100 * acc_oot:.3f}%')

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

Acurácia Treino: 70.158%
Acurácia Teste:  64.741%
Demorou 0.42772865295410156 segundos


## Calculando a probabilidade, predict_proba, na base de Treino e na OOT

In [ ]:
# Calculando a predict_prob
lgbm_model.fit(X_train, y_train)
print(f"ROCAUC (Treino): {100 * roc_auc_score(y_train, lgbm_model.predict_proba(X_train)[:,1]):.3f}%")
print(f"ROCAUC (Teste): {100 * roc_auc_score(y_oot, lgbm_model.predict_proba(X_oot)[:,1]):.3f}%")

ROCAUC (Treino): 78.040%
ROCAUC (Teste): 73.306%


## Grid Search - GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

ti = time.time()
# Criamos um dicionário que os hiperparâmetros que queremos treinar o modelo
# Deve ser o mesmo nome do Pipeline e são dois undelines
# max_depth, learning_rate, n_estimators são parametros do LightGBM
parameters = {
  # Profundidade da árvore.
  'LGBM_Classifier__max_depth': [3, 5, 7, 9],
  # Taxa de aprendizagem (também conhecido como eta) para boosting (entre 0 e 1).
  'LGBM_Classifier__learning_rate': [0.01, 0.015, 0.025, 0.05, 0.1],
  # Número de árvores na floresta aleatório. Possíveis valores: 500, 1000, 2000, 5000, 10000.
  'LGBM_Classifier__n_estimators': [100, 300, 500]
}

# Criamos um GridSearch passando o modelo, os parâmetros, a métrica que queremos otimizar. 
grid_search = GridSearchCV(lgbm_model, parameters, scoring='roc_auc', cv=5, n_jobs=-1)

# Realiza o treinamento com os dados
grid_search.fit(X_train, y_train)

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

Demorou 166.90571188926697 segundos


In [ ]:
# cv_results_ - é um resultado
grid_search.cv_results_

{'mean_fit_time': array([0.28404632, 0.759164  , 1.19948115, 0.37787075, 1.02633262,
        1.67112427, 0.4005825 , 1.084864  , 1.76281209, 0.40021176,
        1.08608246, 1.7541934 , 0.27870746, 0.72843571, 1.14198918,
        0.37498112, 0.9852459 , 1.53813391, 0.39866452, 1.04330168,
        1.66086397, 0.41041651, 1.04014683, 1.64693794, 0.27603121,
        0.70215912, 1.80407257, 0.36976314, 0.90615668, 1.40495   ,
        0.38908887, 0.97206945, 1.5715723 , 0.38515191, 0.95748367,
        1.57308307, 0.265552  , 0.67356381, 1.06823487, 0.34048266,
        0.83790426, 1.35986695, 0.3657649 , 0.90361214, 1.50462174,
        0.37036419, 0.89788756, 1.50657868, 0.24858365, 0.64672241,
        1.06156731, 0.30881867, 0.81717   , 1.31335831, 0.34791245,
        1.53353286, 1.53559275, 0.33160172, 0.86708074, 1.40556679]),
 'mean_score_time': array([0.02858977, 0.07245021, 0.12534289, 0.03695536, 0.10362248,
        0.19989409, 0.04210835, 0.12567739, 0.23282299, 0.04582615,
        0.

In [ ]:
# Criando uma tabela com os resultados do GridSearchCV
results = pd.DataFrame(grid_search.cv_results_)

# Rankeando esses resultados
results.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LGBM_Classifier__learning_rate,param_LGBM_Classifier__max_depth,param_LGBM_Classifier__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
13,0.728436,0.008197,0.088114,0.009700,0.015,3,300,"{'LGBM_Classifier__learning_rate': 0.015, 'LGB...",0.754236,0.752350,0.762806,0.754942,0.760818,0.757031,0.004044,1
36,0.265552,0.008376,0.036684,0.001513,0.05,3,100,"{'LGBM_Classifier__learning_rate': 0.05, 'LGBM...",0.754157,0.752496,0.762798,0.754796,0.760718,0.756993,0.004016,2
2,1.199481,0.012807,0.125343,0.001309,0.01,3,500,"{'LGBM_Classifier__learning_rate': 0.01, 'LGBM...",0.754227,0.752381,0.762802,0.754834,0.760705,0.756990,0.004028,3
25,0.702159,0.018295,0.099547,0.009485,0.025,3,300,"{'LGBM_Classifier__learning_rate': 0.025, 'LGB...",0.754096,0.751818,0.762570,0.754886,0.760431,0.756760,0.004056,4
14,1.141989,0.006289,0.139014,0.002527,0.015,3,500,"{'LGBM_Classifier__learning_rate': 0.015, 'LGB...",0.753953,0.751816,0.762789,0.754705,0.760425,0.756738,0.004155,5
1,0.759164,0.004132,0.072450,0.001830,0.01,3,300,"{'LGBM_Classifier__learning_rate': 0.01, 'LGBM...",0.753618,0.752321,0.762535,0.754502,0.760542,0.756704,0.004057,6
48,0.248584,0.005945,0.037167,0.001967,0.1,3,100,"{'LGBM_Classifier__learning_rate': 0.1, 'LGBM_...",0.753525,0.751617,0.762981,0.754207,0.760377,0.756542,0.004358,7
24,0.276031,0.012978,0.032633,0.001673,0.025,3,100,"{'LGBM_Classifier__learning_rate': 0.025, 'LGB...",0.753132,0.752349,0.762432,0.754141,0.760261,0.756463,0.004086,8
26,1.804073,0.267045,0.219662,0.042147,0.025,3,500,"{'LGBM_Classifier__learning_rate': 0.025, 'LGB...",0.753326,0.751444,0.761677,0.754667,0.760571,0.756337,0.004055,9
27,0.369763,0.004550,0.045648,0.000946,0.025,5,100,"{'LGBM_Classifier__learning_rate': 0.025, 'LGB...",0.752495,0.751456,0.762260,0.754864,0.760126,0.756240,0.004246,10


## Recuperando o melhor AUC

In [ ]:
# Recuperando o melhor AUC
best_results = results.query("index == 13")

In [ ]:
# Listando as colunas da seleção 
best_results.columns.to_list()

['mean_fit_time',
 'std_fit_time',
 'mean_score_time',
 'std_score_time',
 'param_LGBM_Classifier__learning_rate',
 'param_LGBM_Classifier__max_depth',
 'param_LGBM_Classifier__n_estimators',
 'params',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'mean_test_score',
 'std_test_score',
 'rank_test_score']

## Recuperando os melhores Hiperparâmetros

In [ ]:
# Hiperparamentros
hiperparameters = [
  'param_LGBM_Classifier__learning_rate',
  'param_LGBM_Classifier__max_depth',
  'param_LGBM_Classifier__n_estimators',
]

In [ ]:
# Recuperando os melhores hiperparâmetros
best_parameters = {
    'LGBM_Classifier__max_depth': [3],
    'LGBM_Classifier__learning_rate': ['0.015'],
    'LGBM_Classifier__n_estimators': [300]
}

In [ ]:
# Imprimindo os melhores Hiperparametros
best_results[hiperparameters]

,param_LGBM_Classifier__learning_rate,param_LGBM_Classifier__max_depth,param_LGBM_Classifier__n_estimators
13,0.015,3,300


## Avaliando o desempenho do melhor modelo no GridSearchCV na base OOT

In [ ]:
# Avaliando o desempenho do melhor modelo no grid search na base OOT

from sklearn.model_selection import GridSearchCV

ti = time.time()
# Criamos um dicionário que os hiperparâmetros que queremos treinar o modelo
# Deve ser o mesmo nome do Pipeline e são dois undelines
parameters = best_parameters

# Criamos um GridSearch passando o modelo, os parâmetros, a métrica que queremos otimizar. 
best_model = GridSearchCV(lgbm_model, parameters, scoring='roc_auc', cv=5, n_jobs=-1)

# Realiza o treinamento com os dados
best_model.fit(X_oot, y_oot)

# Tempo final
tf = time.time()
print(f'Demorou {tf - ti} segundos')

Demorou 3.5663373470306396 segundos


In [ ]:
# cv_results_ - é um resultado
best_model.cv_results_

{'mean_fit_time': array([0.71179152]),
 'mean_score_time': array([0.07690029]),
 'mean_test_score': array([0.73984098]),
 'param_LGBM_Classifier__learning_rate': masked_array(data=['0.015'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_LGBM_Classifier__max_depth': masked_array(data=[3],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_LGBM_Classifier__n_estimators': masked_array(data=[300],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'LGBM_Classifier__learning_rate': '0.015',
   'LGBM_Classifier__max_depth': 3,
   'LGBM_Classifier__n_estimators': 300}],
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([0.72494683]),
 'split1_test_score': array([0.74257766]),
 'split2_test_score': array([0.73413365]),
 'split3_test_score': array([0.73449509]),
 'split4_test_score': array([0.76305169]),
 'std_fit_time': array([0.10460743]),
 'std_score

In [ ]:
# criando uma tabela com os resultados do GridSearchCV
results = pd.DataFrame(best_model.cv_results_)

# Rankeando esses resultados
results.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LGBM_Classifier__learning_rate,param_LGBM_Classifier__max_depth,param_LGBM_Classifier__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.711792,0.104607,0.0769,0.01842,0.015,3,300,"{'LGBM_Classifier__learning_rate': '0.015', 'L...",0.724947,0.742578,0.734134,0.734495,0.763052,0.739841,0.012878,1


## Calculo da probabilidade da classe 1 para a base OOT - ROC AUC

In [ ]:
# Calculando a probabilidade predict_proba
best_model.fit(X_oot, y_oot)
print(f"ROCAUC (Teste): {100 * roc_auc_score(y_oot, best_model.predict_proba(X_oot)[:,1]):.3f}%")

ROCAUC (Teste): 74.676%
